In [1]:
import os
%load_ext autoreload
%autoreload 2

In [2]:
from torch.utils.data import Dataset
import torch


class MockData(Dataset):
    def __len__(self):
        return 100

    def __getitem__(self, idx):
        return {'input_features': torch.randn(80, 3000), 'speaker_inventory': torch.randn(8, 384),
                'target_asr_ids': torch.randint(0, 60, size=(256,)),
                'target_diar_ids': torch.randint(0, 8, size=(256,))}


data = MockData()
loader = torch.utils.data.DataLoader(data, batch_size=4)

In [3]:
batch = next(iter(loader))

In [2]:
from datasets import load_dataset
from mango.utils.tokenization import retain_cyrillic

cv13 = load_dataset('mozilla-foundation/common_voice_13_0', 'uk', trust_remote_code=True)
urban = load_dataset('danavery/urbansound8K')

urban = urban['train'].rename_column('class', 'label')
cv13 = cv13['train'].rename_columns({'sentence': 'transcription', 'client_id': 'speaker_id'})
cv13 = retain_cyrillic(cv13, 'transcription')

cv13.set_format('pt')
urban.set_format('pt')

In [3]:
from mango.training.SpeakerAttributedMixer import SpeakerAttributeExample, SpeakerAttributedMixer, DatasetMixerConfig

data = SpeakerAttributedMixer(
    DatasetMixerConfig(max_speakers=3, utterances_count=250, beta=5, min_repetitions=3, max_repetitions=6), cv13,
    urban)
example = data.generate()

In [4]:
%load_ext autoreload 
%autoreload 2

In [5]:
from transformers import PreTrainedTokenizerFast

In [6]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('anakib1/sa-asr-0.1')

In [7]:
from mango.training.collators import SpeakerAttributionCollator

In [9]:
from transformers import WhisperFeatureExtractor
import torch

feature_extractor = WhisperFeatureExtractor.from_pretrained('openai/whisper-small')
inventory = torch.randn(30, 384)
collator = SpeakerAttributionCollator(tokenizer, feature_extractor, inventory)

In [10]:
import torch
from mango.training.DatasetMixer import DatasetMixerWrapper

wrapped = DatasetMixerWrapper(data)
loader = torch.utils.data.DataLoader(wrapped, batch_size=4, collate_fn=collator)

In [15]:
tokenizer.pad_token_id

0

In [13]:
from mango.models.sa_asr import SAASR, SAASRConfig

model = SAASR(SAASRConfig(vocab_size=tokenizer.vocab_size))
model(**batch)

{'asr_outputs': tensor([[[ 477.6021,  536.6523,   48.9734,  ...,  392.5928, -523.9099,
             85.6945],
          [ 477.8172,  536.3663,   49.5025,  ...,  392.6454, -523.8537,
             85.5838],
          [ 477.8195,  536.3677,   49.4888,  ...,  392.6532, -523.8458,
             85.5778],
          ...,
          [ 477.8139,  536.3461,   49.4363,  ...,  392.7038, -523.8139,
             85.5769],
          [ 477.8138,  536.3460,   49.4363,  ...,  392.7037, -523.8139,
             85.5768],
          [ 477.8138,  536.3461,   49.4362,  ...,  392.7039, -523.8139,
             85.5769]],
 
         [[ 241.8062,  320.1090,  -11.6099,  ...,  321.1734, -461.2775,
            -35.9849],
          [ 242.0220,  319.7439,  -11.2700,  ...,  321.2599, -461.4782,
            -36.4647],
          [ 242.0175,  319.7428,  -11.2770,  ...,  321.2614, -461.4699,
            -36.4609],
          ...,
          [ 241.9625,  319.7487,  -11.2995,  ...,  321.2768, -461.4478,
            -36.4751],
  

In [ ]:
from mango.training.MangoTrainer import TrainerConfig, MangoTrainer

config = TrainerConfig(model_name='sa-asr-0.1', logs_frequency_batches=16, save_strategy='epoch',
                       mixed_precision='fp16', early_stopping_patience=6, gradient_accumulation_steps=2)

optim = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CyclicLR(optim, base_lr=1e-4, max_lr=1e-3, mode='exp_range', gamma=0.99,
                                              cycle_momentum=False, step_size_up=int(len(loader) * 1.75),
                                              step_size_down=int(len(loader) * 1.25))

trainer = MangoTrainer(model=model, train_loader=loader, eval_loader=loader, config=config, optmizer=optim,
                       scheduler=scheduler)

trainer.train()

In [25]:
a = torch.randint(0, 100, (4, 128))
b = torch.randint(0, 100, (4, 128))
((a==b).masked_fill(b==0, False).sum(dim=1)/((b!=0).sum(dim=1))).mean()

tensor(0.0099)

In [19]:
import evaluate
acc = evaluate.load('accuracy')
acc.compute(references=a, predictions=b)

ValueError: Predictions and/or references don't match the expected format.
Expected format: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)},
Input predictions: tensor([[ 7, 82, 87, 41, 55,  3, 94, 45, 46, 54, 77, 27, 55, 33, 71, 93, 78, 85,
         49, 57, 15, 17,  5, 73, 72, 71, 47,  6, 19, 97,  2, 87, 33, 72, 35, 71,
         89, 43, 51, 28,  7, 80, 85, 70, 73, 20, 57, 60, 92,  3, 73, 81, 25, 92,
         94, 40,  4, 14, 45, 96, 37, 47, 14, 37, 66,  5, 26, 28, 95, 30, 34, 13,
         75, 71, 28, 22, 93, 15, 41, 10, 86, 57, 27, 45, 86, 27, 47, 66, 49, 54,
         61, 63, 96,  5, 65, 62, 10, 46, 92,  4, 45, 66, 20,  8,  4, 65,  9, 62,
         23,  2, 83, 46, 69, 16, 23,  6, 27, 98,  1, 86, 74, 53, 90, 91, 89, 13,
         72, 78],
        [97, 76, 21, 82, 15, 27, 32, 51, 32, 96, 64, 61, 56, 38,  7, 70, 33, 17,
         10, 19, 65, 63, 91, 48, 83, 13, 12, 65, 40, 72, 93, 22, 24, 71, 74, 36,
         80, 77, 47, 89, 22, 75, 44, 40, 74, 70, 